<a href="https://colab.research.google.com/github/j-river1/Webscraping-Elecciones-EstadosUnidos/blob/main/src/WebScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scrapping

Este codigo tiene como finalidad la descarga de los datos de https://ballotpedia.org/Voter_registration

# Instalación de librerias 

Para el webscrapping se utiliza la librerias beatifulSoup [1]

In [ ]:
!pip install beautifulsoup4
#!pip install urllib2


In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml


# Tecnología de la pagina 
En esta sección se muestra la tecnologia utilizada en la pagina web.

In [3]:
pip install builtwith

  Created wheel for builtwith: filename=builtwith-1.3.4-cp36-none-any.whl size=36100 sha256=cbd99eee71ca7f64f6b1e1eb76df39dfdf8de039d6f019dd319b7c92b3698737
  Stored in directory: /root/.cache/pip/wheels/ab/bf/98/1e11b10a7c3bdea99ff8fe79abc7d3d22a254cab4753a88d52
Successfully built builtwith


In [4]:
import builtwith

builtwith.builtwith('https://ballotpedia.org/Voter_registration')


{'analytics': ['Optimizely'],
 'font-scripts': ['Google Font API'],
 'web-frameworks': ['Twitter Bootstrap'],
 'web-servers': ['Apache'],
 'widgets': ['AddThis'],
 'wikis': ['MediaWiki']}

# Introducción

Este web scrapping tiene como objetivo descargar toda la información necesaria para votar en la elecciones presidenciales de los Estados Unidos. El contenido de esta información está en el sitio web **Ballotpedia**. El web scapping empieza por descargar la informacion de las tablas de este sitio web y luego se une una sola base de datos. La base de datos descargada tiene la siguiente información:




* **State**. Estado. 
* **All voters required to show ID**. Todos los votantes deben mostrar una identificación ("Sí" indica que todos los votantes deben presentar una identificación en las urnas). 
* **Select voters required to show ID**. Seleccione los votantes que deben mostrar una identificación (una entrada en este campo describe qué votantes deben mostrar una identificación en las urnas si no hay un requisito de identificación universal). 
* **Accepted IDs** Identificaciones aceptadas (una lista de formas aceptables de identificación). 
*  **Registration URL** URL de registro (un enlace al sitio web del estado donde los votantes elegibles pueden registrarse, si corresponde)

* **Registration status URL** URL de estado de registro (un enlace al sitio web del estado donde los votantes pueden verificar su estado de registro, si corresponde).

* **Registration update URL** URL de actualización de registro (un enlace al sitio web del estado donde los votantes pueden actualizar sus registros, si corresponde)

* **In-person registration deadline** Fecha límite de registro en persona (el último día que un votante puede registrarse en persona, si corresponde).

*  **Mail postmark or receipt deadline**. Fecha límite de registro por correo (el último día que un votante puede registrarse por correo, si corresponde)

* **Mail registration deadline**. Matasellos de correo o fecha límite de recepción (ya sea que la fecha límite de registro por correo sea un matasellos o fecha límite de recepción).

* **Online registration deadline** Fechas límite de registro en línea (el último día que un votante puede registrarse en línea, si corresponde)

* **Same-day registration** Registro el mismo día ("Sí" indica que el estado permite que los votantes se registren el día de las elecciones).

* **Early voting same-day registration** Inscripción anticipada el mismo día ("Sí" indica que el estado permite que los votantes se registren durante el período de votación anticipada, si corresponde).

* **Early voting start date** Fecha de inicio de la votación anticipada

* **Early voting end date** Fecha de finalización de la votación anticipada


A continuación está el codigo para la descarga de cada una de las tablas y de la unión de estas.

# Do I need ID to vote?

In [27]:
URL = "https://ballotpedia.org/Main_Page"
pagina = requests.get(URL)
soup = BeautifulSoup(pagina.content, 'html.parser')
selec = soup.select("a[href*=Voting_in_2020]")
link = selec[0].get('href')
pagina2 = requests.get(link)
soup2 = BeautifulSoup(pagina2.content)


table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="VoterID") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
df = pd.DataFrame(tabla_aux, columns=column_names)
df6 = df[['State', 'All voters required to show ID', 'Select voters required to show ID', 'Accepted IDs']]
df6

,State,All voters required to show ID,Select voters required to show ID,Accepted IDs
0,Nevada,No,"""If a person’s name appears in the election bo...","""(a) The card issued to the voter at the time ..."
1,California,No,"""In most cases, a California voter is not requ...","""A copy of a recent utility bill, the sample b..."
2,Texas,Yes,,"""Texas Driver License issued by the Texas Depa..."
3,Colorado,Yes,,"""A valid Colorado driver’s license or valid id..."
4,American Samoa,Yes,,"""Any qualified elector who wishes to cast his/..."
5,Michigan,Yes,,"""Michigan driver's license or state-issued ID ..."
6,South Carolina,Yes,,"""SC driver's license (includes standard licens..."
7,Kansas,Yes,,"""Driver’s license or ID card issued by Kansas ..."
8,Illinois,No,"""If you register by mail, you must vote in per...","""A current and valid photo identification. Uti..."
9,West Virginia,Yes,,"""Voter registration card; Medicare card or Soc..."


# How do I register to vote?

In [28]:
table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="VoterRegistration") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
print(column_names)
df = pd.DataFrame(tabla_aux, columns=column_names)
df7 = df[['State', 'Registration URL', 'Registration status URL', 'Registration update URL']]
df7


['', 'State', 'Registration URL', 'Registration status URL', 'Registration update URL', '']


,State,Registration URL,Registration status URL,Registration update URL
0,Nevada,https://www.nvsos.gov/sosvoterservices/Registr...,https://www.nvsos.gov/votersearch/,https://www.nvsos.gov/sosvoterservices/Registr...
1,California,https://registertovote.ca.gov/,https://voterstatus.sos.ca.gov/,https://registertovote.ca.gov/
2,Texas,,https://teamrv-mvp.sos.texas.gov/MVP/mvp.do,https://www.votetexas.gov/register-to-vote/did...
3,Colorado,,https://www.sos.state.co.us/voter/pages/pub/ol...,https://www.sos.state.co.us/voter/pages/pub/ol...
4,American Samoa,,,
5,Michigan,https://mvic.sos.state.mi.us/RegisterVoter,https://mvic.sos.state.mi.us/Voter/Index,https://mvic.sos.state.mi.us/RegisterVoter
6,South Carolina,https://info.scvotes.sc.gov/eng/ovr/start.aspx,https://info.scvotes.sc.gov/eng/voterinquiry/V...,https://www.scvotes.gov/update-your-voter-regi...
7,Kansas,https://www.kdor.ks.gov/apps/voterreg/default....,https://myvoteinfo.voteks.org/voterview,https://www.kdor.ks.gov/apps/voterreg/default....
8,Illinois,https://ova.elections.il.gov/,https://ova.elections.il.gov/RegistrationLooku...,https://ova.elections.il.gov/
9,West Virginia,,https://services.sos.wv.gov/Elections/Voter/Am...,https://ovr.sos.wv.gov/Register/Landing#Qualif...


# What is the deadline to register to vote?

In [29]:
table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="RegistrationDeadlines") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
#print(column_names)
df = pd.DataFrame(tabla_aux, columns=column_names)
df8 = df[[ 'State', 'In-person registration deadline', 'Mail registration deadline', 'Mail postmark or receipt deadline', 'Online registration deadline']]
df8

,State,In-person registration deadline,Mail registration deadline,Mail postmark or receipt deadline,Online registration deadline
0,Nevada,"October 6, 2020","October 6, 2020",Postmarked,"October 29, 2020"
1,California,"November 3, 2020","October 19, 2020",Postmarked,"October 19, 2020"
2,Texas,"October 5, 2020","October 5, 2020",Postmarked,
3,Colorado,"November 3, 2020","October 26, 2020",Received,"October 26, 2020"
4,American Samoa,"October 5, 2020",,,
5,Michigan,"November 3, 2020","October 19, 2020",Postmarked,"October 19, 2020"
6,South Carolina,"October 2, 2020","October 5, 2020",Postmarked,"October 4, 2020"
7,Kansas,"October 13, 2020","October 13, 2020",Postmarked,"October 13, 2020"
8,Illinois,"November 3, 2020","October 6, 2020",Postmarked,"October 18, 2020"
9,West Virginia,"October 13, 2020","October 13, 2020",Postmarked,"October 13, 2020"


# Can I register to vote the same day that I vote?

In [30]:
table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="SamedayRegistration") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
df = pd.DataFrame(tabla_aux, columns=column_names)
df9= df[[ 'State', 'Same-day registration ', 'Early voting same-day registration']]
df9

,State,Same-day registration,Early voting same-day registration
0,Nevada,Yes,Yes
1,California,Yes,Yes
2,Texas,No,No
3,Colorado,Yes,Yes
4,American Samoa,No,
5,Michigan,Yes,Yes
6,South Carolina,No,No
7,Kansas,No,No
8,Illinois,Yes,Yes
9,West Virginia,No,No


# Can I vote early?

In [39]:

table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="EarlyVoting") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
print(column_names)
df = pd.DataFrame(tabla_aux, columns=column_names)
df10= df[[ 'State', 'Early voting start date', 'Early voting end date']]
df10

['', 'State', 'Early voting start date', 'Early voting end date', '']


,State,Early voting start date,Early voting end date
0,Nevada,"October 17, 2020","October 30, 2020"
1,California,"October 5, 2020","November 2, 2020"
2,Texas,"October 13, 2020","October 30, 2020"
3,Colorado,"October 19, 2020","November 2, 2020"
4,American Samoa,,
5,Michigan,"September 24, 2020","November 2, 2020"
6,South Carolina,"October 5, 2020","November 2, 2020"
7,Kansas,"October 14, 2020","November 2, 2020"
8,Illinois,"September 24, 2020","November 2, 2020"
9,West Virginia,"October 21, 2020","October 31, 2020"


# Union de las tablas y base de datos final

In [42]:
aux =df9.merge(df8, on='State', how='left')
aux3 = df6.merge(df7, on='State', how='left')
aux4 = aux3.merge(aux, on='State', how='left')
total = aux4.merge(df10, on='State', how='left')

print(total.shape)
total



(56, 15)


,State,All voters required to show ID,Select voters required to show ID,Accepted IDs,Registration URL,Registration status URL,Registration update URL,Early voting start date_x,Early voting end date_x,In-person registration deadline,Mail registration deadline,Mail postmark or receipt deadline,Online registration deadline,Early voting start date_y,Early voting end date_y
0,Nevada,No,"""If a person’s name appears in the election bo...","""(a) The card issued to the voter at the time ...",https://www.nvsos.gov/sosvoterservices/Registr...,https://www.nvsos.gov/votersearch/,https://www.nvsos.gov/sosvoterservices/Registr...,"October 17, 2020","October 30, 2020","October 6, 2020","October 6, 2020",Postmarked,"October 29, 2020","October 17, 2020","October 30, 2020"
1,California,No,"""In most cases, a California voter is not requ...","""A copy of a recent utility bill, the sample b...",https://registertovote.ca.gov/,https://voterstatus.sos.ca.gov/,https://registertovote.ca.gov/,"October 5, 2020","November 2, 2020","November 3, 2020","October 19, 2020",Postmarked,"October 19, 2020","October 5, 2020","November 2, 2020"
2,Texas,Yes,,"""Texas Driver License issued by the Texas Depa...",,https://teamrv-mvp.sos.texas.gov/MVP/mvp.do,https://www.votetexas.gov/register-to-vote/did...,"October 13, 2020","October 30, 2020","October 5, 2020","October 5, 2020",Postmarked,,"October 13, 2020","October 30, 2020"
3,Colorado,Yes,,"""A valid Colorado driver’s license or valid id...",,https://www.sos.state.co.us/voter/pages/pub/ol...,https://www.sos.state.co.us/voter/pages/pub/ol...,"October 19, 2020","November 2, 2020","November 3, 2020","October 26, 2020",Received,"October 26, 2020","October 19, 2020","November 2, 2020"
4,American Samoa,Yes,,"""Any qualified elector who wishes to cast his/...",,,,,,"October 5, 2020",,,,,
5,Michigan,Yes,,"""Michigan driver's license or state-issued ID ...",https://mvic.sos.state.mi.us/RegisterVoter,https://mvic.sos.state.mi.us/Voter/Index,https://mvic.sos.state.mi.us/RegisterVoter,"September 24, 2020","November 2, 2020","November 3, 2020","October 19, 2020",Postmarked,"October 19, 2020","September 24, 2020","November 2, 2020"
6,South Carolina,Yes,,"""SC driver's license (includes standard licens...",https://info.scvotes.sc.gov/eng/ovr/start.aspx,https://info.scvotes.sc.gov/eng/voterinquiry/V...,https://www.scvotes.gov/update-your-voter-regi...,"October 5, 2020","November 2, 2020","October 2, 2020","October 5, 2020",Postmarked,"October 4, 2020","October 5, 2020","November 2, 2020"
7,Kansas,Yes,,"""Driver’s license or ID card issued by Kansas ...",https://www.kdor.ks.gov/apps/voterreg/default....,https://myvoteinfo.voteks.org/voterview,https://www.kdor.ks.gov/apps/voterreg/default....,"October 14, 2020","November 2, 2020","October 13, 2020","October 13, 2020",Postmarked,"October 13, 2020","October 14, 2020","November 2, 2020"
8,Illinois,No,"""If you register by mail, you must vote in per...","""A current and valid photo identification. Uti...",https://ova.elections.il.gov/,https://ova.elections.il.gov/RegistrationLooku...,https://ova.elections.il.gov/,"September 24, 2020","November 2, 2020","November 3, 2020","October 6, 2020",Postmarked,"October 18, 2020","September 24, 2020","November 2, 2020"
9,West Virginia,Yes,,"""Voter registration card; Medicare card or Soc...",,https://services.sos.wv.gov/Elections/Voter/Am...,https://ovr.sos.wv.gov/Register/Landing#Qualif...,"October 21, 2020","October 31, 2020","October 13, 2020","October 13, 2020",Postmarked,"October 13, 2020","October 21, 2020","October 31, 2020"


# Base de datos final

# Descarga de la pagina web

In [5]:
import re
import pandas as pd

def descarga_objetos (URL):
  '''
  Esta función sirve para descargar la informacion de la pagina web

  Input :  URL es la direccion de la pagina donde se va a descargar la 
           informacion.

            El tipo de objetos. Debe seleccionar una de las siguientes 
            opciones dependiendo del tipo de objeto:

            1 = El tipo de objeto tag.
            2



  Output : Devuelve el contenido de la descarga
  
  '''
  #URL = "http://socialmention.com/search?q=trump&t=all&btnG=Search"
 
  URL = "https://ballotpedia.org/Voter_registration"
  soup = BeautifulSoup(pagina.content) 
  return  soup.prettify()

#def tipo_objetos (URL)


#URL = "https://ballotpedia.org/Voter_registration"
URL = "https://ballotpedia.org/Main_Page"
#URL = "http://socialmention.com/search?q=trump&t=all&btnG=Search"
pagina = requests.get(URL)
soup = BeautifulSoup(pagina.content, 'html.parser')
#selec = soup.select("a[href*=Voting_in]")
selec = soup.select("a[href*=Voting_in_2020]")
#print()
#print(type(selec))
#print(type(selec[0]))
link = selec[0].get('href')
pagina2 = requests.get(link)
#print(selec[0].get('href'))
#print(pagina2)
soup2 = BeautifulSoup(pagina2.content)

#for i in range(len(selec)):
  #total = URL + selec[i].get('href')
  #print(total)
  #print(type(selec[i]))
  #print((selec[i].name))
  #print(selec[i].get('href'))
  #print(type(selec[i].get('href')))


#print(selec[0])
#print(soup.select("a[href*=Voting_in]"))
#print(len(soup.select("a[href*=Voting_in]")))

#print(soup.prettify())
print(soup2.prettify())

#print(soup.title)
#print(soup.title.name)
#print(soup.title.parent.name)
#print(soup.p)
#print(soup.p['class'])
#print(soup.a)
#print(soup.find_all('a'))
#print()
#print()
#print()

#print(soup.prettify())
#for link in soup2.find_all('a'):
  #we = URL + link.get('href')
  #print(we)
  #print(type(link.get('href')))
  #print(link)
  #print((link.get('href')))

#for tag in soup.find_all(re.compile('t')):
  #we = URL + link.get('href')
  #print(we)
  #print(type(link.get('href')))
  #print(link)
  #print((link.get('href="/Voting_')))
  #print(tag)
  #print(tag.name)

#table = soup2.find(lambda tag: tag.name=='display compact cell-border responsive dtable' and tag.has_attr('id') and tag['id']=="VoterID") 
table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="AbsenteeReturn") 
rows = table.findAll(lambda tag: tag.name=='tr')


#table = soup.find(lambda tag: tag.name=='table') 
#print(table)
#rows = table.findAll(lambda tag: tag.name=='tr')




#print(type(rows))
tabla = []
for i in range(len(rows)):
  #tabla.append(str(rows[i]))
  #tabla.append(rows[i].text)
  tabla.append(rows[i].get_text())
  #tabla.append(rows[i].text.replace('S', 'q'))  
  #tabla.append(rows[i].astype('str')) 
  #tabla.append(rows[i].astype('str'))
  
  #print(type(rows[i].name))
  #print(rows[i].attrs)
  #print(type(rows[i].text))

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))
  #tabla_aux[j].replace(" ", "") 
  #tabla_aux[j].remove("") 
  #tabla_aux[j] = ''.join(tabla_aux[j]).split() 

tabla_aux2 = []
#for j in range(len(tabla_aux)):
  #tabla_aux.append(tabla[j].split("\n"))
 # tabla_aux2[j].append(tabla_aux[j].replace(" ", ""))
  #print(type(tabla_aux[[j]]))
  #print(len(tabla_aux[j]))



#salida = [item for item in tabla_aux if len(item)==1] 
#print(salida)
#tabla_aux2 = 
column_names = tabla_aux.pop(0)
#print(column_names)
df = pd.DataFrame(tabla_aux, columns=column_names)
#column_names = tabla_aux.pop(0)
print(column_names)
#print(df)

#df1 = df[['State', 'Notary or witness requirements', 'Signature requirements']]

#df1 = df[['State', 'In-person return deadline', 'Mail return deadline', 'Mail postmark or receipt deadline']]


print(df1)
#print(df)
#print(df1.shape)
#print(df.columns)

#for col in df1.columns: 
    #print(col) 

#print(tabla_aux[1][3])
#print(type(tabla[2]))  
#print(type(tabla ))
#print(tabla[0].split("\n"))

#print(tabla_aux)
#string = []
#for i in range(len(tabla)):
#  string.append(tabla[i].astype('str'))


  
  
  #print(type(rows[i].strings))
  
  #print(rows[i]

#table = soup.find_all('table')
#AbsenteeWitness
#table = soup.find_all('AbsenteeWitness')

#df = pd.read_html(str(table))[0]

#df = pd.read_html(str(rows))[0]
#df = pd.DataFrame(tabla, columns= [1,2,3,4,5,6,7])


#print(len(tabla))
#print(type(tabla[0]))
#print(tabla[1])
#https://shaynemei.github.io/hackore/bs4_basic.html
#seven_day = soup2.find(id="AbsenteeWitness" ).get_text()

#print(type(seven_day[0]))
#print(len(seven_day))
#print(type(seven_day))
#print(seven_day)
#print(seven_day[1])

#df = pd.DataFrame(tabla, columns= tabla[0])
#df[0].apply(lambda x: (re.sub(r'<.*?>', '', str(x))))

#print(df.columns)

#df = pd.DataFrame(tabla)
#df1 = df[[2, 6]]
#df1 = df[[1, 5]]
#df1=df.loc[ [1]] 
#df1[1] = df1[1].astype('str')

#df1[[2]] = df1[[2]].astype('str')
#df1[2] = df1[2].str.replace('<th>', 'q')
#df1[1].replace("State", "")


#print(type(df[1]))
#print(df1)
#print(df)
#print(df.shape)
#print(df.infer_objects)
#print(df.dtypes)
#print(df[[1]])
#print(df.columns)
#print(df[2])

#print(df.shape)
#print(type(df1[1]))
#print(df1)
#df1.to_csv('q.csv')


#dataTypeSeries = df1.dtypes
#print(dataTypeSeries)

#print(type(df1[2]))
#print(df1[1])
#print(df1.columns)
#print(df1[1].astype('str'))
#print(type(df1[1]))

#print(tabla)

#print(df)
#print(type(table))
#print(table.name)
#print(rows[0])
#print(len(rows))
#print(type(table))
#print((table))
#print(type(rows))
#print(rows)
#rows = table.findAll(lambda tag: tag.name=='tr')

#print(result)
#print(soup.get_text())
#html_page = urllib2.urlopen(URL)
#soup = BeautifulSoup(html_page)

#for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
#    print(link.get('href'))


#soup_b  = BeautifulSoup('<b class="boldest">Extremely bold</b>')
#tag = soup_b.b
#type(tag)
#print(tag.name)
#print(tag.attrs)
#print(tag['id'])
#soup = BeautifulSoup(page.content)
#import pandas as pd
#url = r'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
#tables = pd.read_html(url) # Returns list of all tables on page
#tables = pd.read_html(link) # Returns list of all tables on page

#sp500_table = tables[0] # Select table of interest
#print(sp500_table )
#print(pagina2)
#print(link)
#print(sp500_table)




#soup2.findAll('table')[0].tbody.findAll('tr')














<!-- here -->
<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Voting in 2020 - Ballotpedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Voting_in_2020","wgTitle":"Voting in 2020","wgCurRevisionId":8177510,"wgRevisionId":8177510,"wgArticleId":1107416,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Voting policy concepts and issues","2020 elections"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","Apri

# Do I need to sign or have a witness for my absentee ballot?

In [2]:

URL = "https://ballotpedia.org/Main_Page"
pagina = requests.get(URL)
soup = BeautifulSoup(pagina.content, 'html.parser')
selec = soup.select("a[href*=Voting_in_2020]")
link = selec[0].get('href')
pagina2 = requests.get(link)
soup2 = BeautifulSoup(pagina2.content)

table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="AbsenteeWitness") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
df = pd.DataFrame(tabla_aux, columns=column_names)
df1 = df[['State', 'Notary or witness requirements', 'Signature requirements']]
df1

,State,Notary or witness requirements,Signature requirements
0,Nevada,No requirement,Signature required.
1,California,No requirement,Signature required on envelope.
2,Texas,No requirement,Signature required.
3,Colorado,No requirement,Signature required on envelope.
4,American Samoa,No requirement,Not specified
5,Michigan,No requirement,Signature required on envelope.
6,South Carolina,Witness only,Signature required.
7,Kansas,No requirement,Signature required on envelope.
8,Illinois,No requirement,Signature required on envelope.
9,West Virginia,No requirement,Signature required on envelope.


# What is the deadline to return an absentee ballot?

In [3]:
table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="AbsenteeReturn") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
df = pd.DataFrame(tabla_aux, columns=column_names)
df2 = df[['State', 'In-person return deadline', 'Mail return deadline', 'Mail postmark or receipt deadline']]
df2

,State,In-person return deadline,Mail return deadline,Mail postmark or receipt deadline
0,Nevada,"November 3, 2020","November 3, 2020",Postmarked
1,California,"November 3, 2020","November 3, 2020",Postmarked
2,Texas,"November 3, 2020","November 3, 2020",Postmarked
3,Colorado,"November 3, 2020","November 3, 2020",Received
4,American Samoa,"November 3, 2020","November 3, 2020",Received
5,Michigan,"November 3, 2020","November 3, 2020",Received
6,South Carolina,"November 3, 2020","November 3, 2020",Received
7,Kansas,"November 3, 2020","November 3, 2020",Postmarked
8,Illinois,"November 3, 2020","November 3, 2020",Postmarked
9,West Virginia,"November 2, 2020","November 3, 2020",Postmarked


# How do I request an absentee ballot?

In [4]:
table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="AbsenteeRequest") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
df = pd.DataFrame(tabla_aux, columns=column_names)
df3 = df[['State', 'Is other documentation required?', 'Required documentation', 'Source', 'Request URL']]
df3

,State,Is other documentation required?,Required documentation,Source,Request URL
0,Nevada,No,,https://www.nvsos.gov/sos/elections/voters/abs...,https://www.nvsos.gov/sos/elections/voters/abs...
1,California,No,,https://www.sos.ca.gov/elections/voter-registr...,https://www.sos.ca.gov/elections/voter-registr...
2,Texas,No,,https://www.sos.texas.gov/elections/voter/reqa...,https://www.sos.texas.gov/elections/voter/reqa...
3,Colorado,No,,https://www.sos.state.co.us/pubs/elections/FAQ...,https://www.sos.state.co.us/voter/pages/pub/ol...
4,American Samoa,Yes,An individual who is voting by absentee ballot...,https://aselectionoffice.gov/node/6,https://aselectionoffice.gov/node/6
5,Michigan,No,,"https://www.michigan.gov/sos/0,4670,7-127-1633...",https://mvic.sos.state.mi.us/avapplication
6,South Carolina,,,https://www.scvotes.gov/absentee-voting,https://info.scvotes.sc.gov/eng/voterinquiry/V...
7,Kansas,Yes,If a voter is not able to provide a valid driv...,https://sos.ks.gov/elections/elections-faq.htm...,https://sos.ks.gov/elections/voter-information...
8,Illinois,No,,https://elections.il.gov/electionoperations/Vo...,https://elections.il.gov/ElectionOperations/Vo...
9,West Virginia,No,,https://sos.wv.gov/elections/Pages/AbsenteeVot...,https://sites.omniballot.us/54/absentee/app/home


# What is the deadline to request an absentee ballot?

In [7]:
table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="AbsenteeDeadlines") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
df = pd.DataFrame(tabla_aux, columns=column_names)
df4 = df[['State', 'In-person request deadline', 'Online request deadline', 'Mail request deadline', 'Mail postmark or receipt deadline']]
df4

['', 'State', 'In-person request deadline', 'Online request deadline', 'Mail request deadline', 'Mail postmark or receipt deadline', '']


,State,In-person request deadline,Online request deadline,Mail request deadline,Mail postmark or receipt deadline
0,Nevada,"October 20, 2020",,"October 20, 2020",Received
1,California,"November 3, 2020",,"October 27, 2020",Received
2,Texas,"October 12, 2020",,"October 23, 2020",Received
3,Colorado,"October 26, 2020",,"October 26, 2020",Received
4,American Samoa,"November 2, 2020",,"October 19, 2020",Received
5,Michigan,"November 2, 2020","October 30, 2020","October 30, 2020",Received
6,South Carolina,"October 30, 2020",,"October 24, 2020",Received
7,Kansas,"October 27, 2020",,"October 27, 2020",Received
8,Illinois,"November 2, 2020","October 29, 2020","October 29, 2020",Received
9,West Virginia,"October 28, 2020","October 28, 2020","October 28, 2020",Received


# Am I eligible to vote absentee?

In [10]:
table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="AbsenteeEligibility") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
#print(column_names)
df = pd.DataFrame(tabla_aux, columns=column_names)
df5 = df[['State', 'Are there eligibility limits?', 'Were changes made due to COVID-19?', 'COVID-19 changes', 'Source']]
df5

,State,Are there eligibility limits?,Were changes made due to COVID-19?,COVID-19 changes,Source
0,Nevada,No,Yes,Absentee/mail-in ballots sent automatically to...,https://www.nvsos.gov/sos/elections/voters/abs...
1,California,No,Yes,Absentee/mail-in ballots sent automatically to...,https://www.sos.ca.gov/elections/voter-registr...
2,Texas,Yes,No,,https://www.sos.texas.gov/elections/voter/reqa...
3,Colorado,No,No,,https://www.sos.state.co.us/pubs/elections/FAQ...
4,American Samoa,Yes,Yes,Absentee/mail-in voting eligibility extended t...,https://aselectionoffice.gov/node/29
5,Michigan,No,Yes,Absentee/mail-in ballot applications sent auto...,"https://www.michigan.gov/sos/0,4670,7-127-1633..."
6,South Carolina,No,Yes,Absentee/mail-in voting eligibility extended t...,https://www.scvotes.gov/absentee-voting
7,Kansas,No,No,,https://sos.ks.gov/elections/elections-faq.htm...
8,Illinois,No,Yes,Absentee/mail-in ballot applications sent auto...,https://elections.il.gov/electionoperations/Vo...
9,West Virginia,No,Yes,Absentee/mail-in ballot eligibility in the gen...,https://sos.wv.gov/elections/Pages/AbsenteeVot...


id="VoterID"

# Referencias

[1] https://www.crummy.com/software/BeautifulSoup/bs4/doc/#quick-start